### 2000~2025的美联储利率

In [ ]:
import requests
import pandas as pd

## https://fredaccount.stlouisfed.org/apikeys
api_key = '2799e9ff56082e330f1d816ae93a1f87'
series_id = 'EFFR'
url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    observations = data['observations']
    
    # 转成 DataFrame
    df = pd.DataFrame(observations)
    
    # 只保留日期和数值，去除无效数据
    df_bitget_bitget = df[['date', 'value']]
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df.dropna(inplace=True)
    
    # 保存为 CSV 文件
    df.to_csv('fed_funds_rate.csv', index=False)
    print("数据已保存为 fed_funds_rate.csv")
    print(df.head())
else:
    print("请求失败，状态码:", response.status_code)

### 处理具体加息降息的时间点

In [2]:
import requests
import pandas as pd

series_id1 = 'DFEDTARU'  # 选择目标利率上限，或换成 DFEDTARL / DFEDTAR
series_id2 = 'DFEDTARL'

# 请求数据
url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id1}&api_key={api_key}&file_type=json'
response = requests.get(url)
data = response.json()
observations = data['observations']
df_upper = pd.DataFrame(observations)

url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id2}&api_key={api_key}&file_type=json'
response = requests.get(url)
data = response.json()
observations = data['observations']
df_lower = pd.DataFrame(observations)

# 数据处理
df_upper['date'] = pd.to_datetime(df_upper['date'])
df_lower['date'] = pd.to_datetime(df_lower['date'])
df_upper.rename(columns={'value': 'upper'}, inplace=True)
df_lower.rename(columns={'value': 'lower'}, inplace=True)
df = pd.merge(df_lower, df_upper, on='date', how='inner')
df = df[['date','lower','upper']]

df['upper'] = pd.to_numeric(df['upper'], errors='coerce')
df['lower'] = pd.to_numeric(df['lower'], errors='coerce')
df.dropna(inplace=True)

# 识别加息/降息
df['change'] = df['upper'] - df['upper'].shift(1)
df['event'] = df['change'].apply(lambda x: '加息' if x > 0 else ('降息' if x < 0 else '无变化'))

# 仅筛选加息或降息日期
rate_change_df = df[df['event'] != '无变化'].copy()

# 打印前几行示例
print(rate_change_df[['date', 'lower', 'upper', 'change', 'event']].head())

# 保存结果
rate_change_df.to_csv('fed_rate_change_history.csv', index=False)
print("加息/降息历史数据已保存为 fed_rate_change_history.csv")


           date  lower  upper  change event
2556 2015-12-16   0.25   0.50    0.25    加息
2920 2016-12-14   0.50   0.75    0.25    加息
3012 2017-03-16   0.75   1.00    0.25    加息
3103 2017-06-15   1.00   1.25    0.25    加息
3285 2017-12-14   1.25   1.50    0.25    加息
加息/降息历史数据已保存为 fed_rate_change_history.csv


### 加密市场的恐慌贪婪指数

In [6]:
# https://alternative.me/crypto/fear-and-greed-index/#google_vignette
from datetime import datetime

start_date = datetime(2018,2,1)
end_date = datetime.today()
days = (end_date - start_date).days

url = f'https://api.alternative.me/fng/?limit={days-1}'
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data['data'])

df['date'] = pd.to_datetime(df['timestamp'], unit='s')
df = df.sort_values('date', ascending=True).reset_index(drop=True)
df = df[['date','value','value_classification']]
print(df.head())
df.to_csv('fear_greed_history_sorted.csv', index=False)
print("已保存为 fear_greed_history_sorted.csv")

        date value value_classification
0 2018-02-01    30                 Fear
1 2018-02-02    15         Extreme Fear
2 2018-02-03    40                 Fear
3 2018-02-04    24         Extreme Fear
4 2018-02-05    11         Extreme Fear
已保存为 fear_greed_history_sorted.csv


C:\Users\looik\AppData\Local\Temp\ipykernel_33440\2944143835.py:13: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['date'] = pd.to_datetime(df['timestamp'], unit='s')


### 获取BTC和ETH的历史价格

In [7]:
# TokenInsight
def getTokenPrice(name:str):
    api_key_tokeninsight = "e401311c82004971b85481f6926bccc2"

    url = f"https://api.tokeninsight.com/api/v1/history/coins/{name}?interval=day&length=-1"

    headers = {
        "accept": "application/json",
        "TI_API_KEY": api_key_tokeninsight
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    data = data['data']
    file_name = data['name'] + "_price.csv"

    df = pd.DataFrame(data["market_chart"])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.sort_values('date', ascending=True).reset_index(drop=True)
    df = df[["date","price","market_cap","vol_spot_24h"]]
    # df = df.dropna(axis=0) 
    print(df.head())
    df.to_csv(file_name, index=False)


getTokenPrice(name="bitcoin")
getTokenPrice(name="ethereum")



        date     price  market_cap  vol_spot_24h
0 2010-07-18  0.085840         NaN           NaN
1 2010-07-19  0.080800         NaN           NaN
2 2010-07-20  0.074736         NaN           NaN
3 2010-07-21  0.079193         NaN           NaN
4 2010-07-22  0.058470         NaN           NaN
        date     price    market_cap  vol_spot_24h
0 2015-08-07  2.831620           NaN  9.062200e+04
1 2015-08-08  1.330750  8.033948e+07  3.680700e+05
2 2015-08-10  0.687586  4.155631e+07  4.004641e+05
3 2015-08-11  1.067379  6.453901e+07  1.518998e+06
4 2015-08-12  1.256613  7.601326e+07  2.073893e+06


### 处理BTC、ETH和其他加密货币的市值

In [8]:
# TokenInsight
df = pd.read_csv("./unprocessedFile/top-coins-day-data-all-tokeninsight-dashboard.csv")
df = df.dropna(axis=0).reset_index(drop=True)
df['All'] = df['BTC Market Cap'] + df['ETH Market Cap'] + df['Top 10 (Without BTC & ETH)'] + df['Others']
df.to_csv("marketCap.csv", index=False)
df.head()

,Date,BTC Market Cap,ETH Market Cap,Top 10 (Without BTC & ETH),Others,All
0,2015-08-07,4.015651e+09,0.000000e+00,4.846071e+08,3.398211e+07,4.534241e+09
1,2015-08-08,3.745911e+09,4.548690e+07,4.796699e+08,7.297106e+07,4.344039e+09
2,2015-08-09,3.811338e+09,4.239955e+07,5.200487e+08,1.556015e+08,4.529388e+09
3,2015-08-10,3.817350e+09,4.155631e+07,4.587565e+08,7.142085e+07,4.389084e+09
4,2015-08-11,3.909601e+09,6.453901e+07,4.689033e+08,6.843116e+07,4.511474e+09


In [31]:
import json
with open('./unprocessedFile/TotalCryptoMarketcap.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

dfs = []

for indicator in data['data']:
    raw_data = json.loads(indicator['data'])
    df = pd.DataFrame(raw_data, columns=['timestamp', indicator['name']])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    df[indicator['name']] = pd.to_numeric(df[indicator['name']], errors='coerce')
    df.set_index('timestamp', inplace=True)
    dfs.append(df)

df_result = pd.concat(dfs, axis=1)
## df_result = df_result.fillna(0)
df_result.columns = ['加密货币总市值','加密货币24h总成交量','山寨币总市值','山寨币24h总成交量']
df_result['BTC总市值'] = df_result['加密货币总市值'] - df_result['山寨币总市值']
df_result['BTC 24h总交易量'] = df_result['加密货币24h总成交量'] - df_result['山寨币24h总成交量']
##df_result.to_csv("crypto_ETF_Data.csv")
df_result.head()

,加密货币总市值,加密货币24h总成交量,山寨币总市值,山寨币24h总成交量,BTC总市值,BTC 24h总交易量
timestamp,,,,,,
2013-04-28,1.583440e+09,0.0,NaN,NaN,NaN,NaN
2013-04-29,1.686950e+09,0.0,86409766.0,0.0,1.600540e+09,0.0
2013-04-30,1.637390e+09,0.0,91107558.0,0.0,1.546282e+09,0.0
2013-05-01,1.333880e+09,0.0,79753553.0,0.0,1.254127e+09,0.0
2013-05-02,1.275410e+09,0.0,72095693.0,0.0,1.203314e+09,0.0


### 处理活跃地址数据

In [9]:
# Token Terminal
df = pd.read_csv("./unprocessedFile/ActiveAddresses(daily).csv")

## Bitcoin active address
df_bitcoin = df[['Date','Bitcoin']].copy()
df_bitcoin

## Ethereum active address
df_Ethereum = df[['Date','Ethereum']].copy()
df_Ethereum = df_Ethereum.dropna()

## Top 10(Without BTC & ETH)
df_others = df.drop(columns=['Bitcoin','Ethereum'])
df_others = df_others.fillna(0)
df_others['Total'] = df_others[df_others.columns[1:]].sum(axis=1)
df_others = df_others[df_others['Total']!=0]
df_others = df_others[['Date','Total']]

df = df_bitcoin.merge(df_Ethereum, on='Date', how='outer').merge(df_others, on='Date', how='outer')
df = df.dropna(axis=0).reset_index(drop=True)
df.to_csv("active_address.csv",index=False)
df

,Date,Bitcoin,Ethereum,Total
0,2018-06-25,431947.0,364086.0,372.0
1,2018-06-26,415627.0,259678.0,507.0
2,2018-06-27,410763.0,248747.0,510.0
3,2018-06-28,381017.0,240925.0,432.0
4,2018-06-29,377532.0,255961.0,344.0
...,...,...,...,...
2447,2025-03-07,614005.0,396118.0,13172465.0
2448,2025-03-08,450898.0,342903.0,11702074.0
2449,2025-03-09,452310.0,342109.0,11524248.0
2450,2025-03-10,546981.0,394804.0,12205917.0


### 爬取Sosovalues上的ETF数据（BTC&ETH 美国ETF现货）

In [ ]:
# https://gw.sosovalue.com/data/s-chart-config-do/findByName
# https://sosovalue.com/zh/assets/etf/Total_Crypto_Spot_ETF_Fund_Flow?page=usBTC

with open('./unprocessedFile/cryptoETF.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

target_names = ["Etf_btc_Fund_flow", "Etf_btc_Total_Net_Assets_Value", "Etf_us_eth_Fund_flow", "Etf_us_eth_Total_Net_Assets_Value"]

dfs = []

for indicator in data['indicatorDataList']:
    if indicator['name'] in target_names:
        raw_data = json.loads(indicator['data'])
        df = pd.DataFrame(raw_data, columns=['timestamp', indicator['name']])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[indicator['name']] = pd.to_numeric(df[indicator['name']], errors='coerce')
        df.set_index('timestamp', inplace=True)
        dfs.append(df)

df_result = pd.concat(dfs, axis=1)
df_result = df_result.fillna(0)
df_result.columns = ['BTC ETF净流入流出','BTC ETF 总净资产价值','ETH ETF净流入流出','ETH ETF 总净资产价值']
print(df_result.head())
df_result.to_csv("crypto_ETF_Data.csv")

            BTC ETF净流入流出  BTC ETF 总净资产价值  ETH ETF净流入流出  ETH ETF 总净资产价值
timestamp                                                             
2024-01-10  0.000000e+00    2.871028e+10           0.0             0.0
2024-01-11  6.280696e+08    2.937523e+10           0.0             0.0
2024-01-12  1.909738e+08    2.793277e+10           0.0             0.0
2024-01-16 -8.058183e+07    2.753279e+10           0.0             0.0
2024-01-17  5.175031e+08    2.771607e+10           0.0             0.0


### 各交易所的BTC资金费率

In [ ]:
# BINANCE（Website）
df_binance = pd.read_csv("./unprocessedFile/Binance Funding Rate History_BTCUSDT.csv")
df_binance = df_binance[['Time','Funding Rate']]
df_binance = df_binance.sort_values('Time', ascending=True).reset_index(drop=True)
df_binance

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\looik\AppData\Local\Temp\ipykernel_33440\2274446224.py:2: SyntaxWarning: invalid escape sequence '\B'
  df_binance = pd.read_csv("./unprocessedFile\Binance Funding Rate History_BTCUSDT.csv")


,Time,Funding Rate
0,2019-09-10 16:00:00,0.010000%
1,2019-09-11 00:00:00,0.010000%
2,2019-09-11 08:00:00,0.010000%
3,2019-09-11 16:00:00,0.010000%
4,2019-09-12 00:00:00,0.010000%
...,...,...
6028,2025-03-12 00:00:00,0.008746%
6029,2025-03-12 08:00:00,0.000576%
6030,2025-03-12 16:00:00,0.005815%
6031,2025-03-13 00:00:00,0.004013%


In [ ]:
# BYBIT（Website + library）
from pybit.unified_trading import HTTP
import time
df_bybit = pd.read_csv("./unprocessedFile/Bybit funding_rate_history_BTCUSDT.csv")
df_bybit = df_bybit[['Time','Funding Rate']]
df_bybit['Time'] = pd.to_datetime(df_bybit['Time']) 
df_bybit['Funding Rate'] = df_bybit['Funding Rate'].astype(float)
df_bybit = df_bybit.sort_values('Time', ascending=True).reset_index(drop=True)

start_date = int(datetime(2025,2,25).timestamp() * 1000)
end_date = int(datetime.today().timestamp() * 1000)
limit = 200
session = HTTP()
current_start = start_date
data_bybit = []

while current_start < end_date:
    response = session.get_funding_rate_history(
        category="linear",
        symbol="BTCPERP",
        startTime = current_start,
        endTime = end_date,
        limit = limit
    )
    result = response.get('result', {}).get('list', [])
    if not result:
        break

    data_bybit.extend(result) 
    last_time = int(result[0]['fundingRateTimestamp'])
    current_start = last_time + 1 

    time.sleep(0.2)

df_bybit2 = pd.DataFrame(data_bybit)
df_bybit2['fundingRateTimestamp'] = pd.to_datetime(df_bybit2['fundingRateTimestamp'],unit='ms').reset_index(drop=True)
df_bybit2 = df_bybit2.drop(columns='symbol')
df_bybit2 = df_bybit2[['fundingRateTimestamp','fundingRate']]
df_bybit2['fundingRate'] = df_bybit2['fundingRate'].astype(float)
df_bybit2.columns = ['Time', 'Funding Rate']
df_bybit2 = df_bybit2.reset_index(drop=True) 

df_bybit = pd.concat([df_bybit, df_bybit2], axis=0, ignore_index=True)
df_bybit = df_bybit.sort_values('Time', ascending=True).reset_index(drop=True)
df_bybit

C:\Users\looik\AppData\Local\Temp\ipykernel_33440\1103906409.py:36: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_bybit2['fundingRateTimestamp'] = pd.to_datetime(df_bybit2['fundingRateTimestamp'],unit='ms').reset_index(drop=True)


,Time,Funding Rate
0,2020-03-25 16:00:00,0.000100
1,2020-03-26 00:00:00,0.000100
2,2020-03-26 08:00:00,0.000100
3,2020-03-26 16:00:00,0.000100
4,2020-03-27 00:00:00,0.000100
...,...,...
5439,2025-03-12 00:00:00,0.000064
5440,2025-03-12 08:00:00,0.000089
5441,2025-03-12 16:00:00,0.000075
5442,2025-03-13 00:00:00,0.000079


In [ ]:
# BITGET（API）
dfs = []
for i in range(1,43):
    url = "https://api.bitget.com/api/v2/mix/market/history-fund-rate"
    params = {
        "symbol":"BTCUSDT",
        "productType": "usdt-futures",
        "pageSize":100,
        "pageNo":i
    }
    response = requests.get(url, params=params)
    data_bitget = response.json()
    dfs.append(data_bitget['data'])
    
dfs = [item for i in dfs for item in i]
df_bitget = pd.DataFrame(dfs)
df_bitget['fundingTime'] = pd.to_datetime(df_bitget['fundingTime'], unit='ms')
df_bitget = df_bitget.sort_values('fundingTime', ascending=True).reset_index(drop=True)
df_bitget = df_bitget.drop(columns=['symbol'])
df_bitget = df_bitget[['fundingTime','fundingRate']]
df_bitget


[{'symbol': 'BTCUSDT', 'fundingRate': '0.000011', 'fundingTime': '1741824000000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.00004', 'fundingTime': '1741795200000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000066', 'fundingTime': '1741766400000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000014', 'fundingTime': '1741737600000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000106', 'fundingTime': '1741708800000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000037', 'fundingTime': '1741680000000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000052', 'fundingTime': '1741651200000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000055', 'fundingTime': '1741622400000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000018', 'fundingTime': '1741593600000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000056', 'fundingTime': '1741564800000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000124', 'fundingTime': '1741536000000'}, {'symbol': 'BTCUSDT', 'fundingRate': '0.000042', 'fundingTime': '1741507200000'}, {'symbol': 'BTCU

C:\Users\looik\AppData\Local\Temp\ipykernel_33440\1760217807.py:19: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_bitget['fundingTime'] = pd.to_datetime(df_bitget['fundingTime'], unit='ms')


,fundingTime,fundingRate
0,2021-05-18 15:00:00,0
1,2021-05-18 23:00:00,0
2,2021-05-19 07:00:00,0.0005
3,2021-05-19 15:00:00,-0.00031
4,2021-05-19 23:00:00,0.0005
...,...,...
4179,2025-03-11 16:00:00,0.000106
4180,2025-03-12 00:00:00,0.000014
4181,2025-03-12 08:00:00,0.000066
4182,2025-03-12 16:00:00,0.00004


In [ ]:
# OKX BitMex


,DateTime,OKEx,BitMEX
0,2020-02-09,0.000420,0.000992
1,2020-02-10,0.000480,0.000100
2,2020-02-11,0.000135,0.000491
3,2020-02-12,0.000700,0.000556
4,2020-02-13,0.000678,0.000722
...,...,...,...
2041,2025-03-01,-0.000001,0.000100
2042,2025-03-02,-0.000052,0.000113
2043,2025-03-03,0.000015,0.000186
2044,2025-03-04,-0.000067,0.000180


### BTC、ETH和加密货币的净流入流出量

In [189]:
# 一周为单位（还在找着有没有一天的）
# IntoTheBlock
with open('./unprocessedFile/crypto_exchange_netflows.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def crypto_netflow(token_name:str):

    df = pd.DataFrame(data[token_name]['metric'])
    df = df[[0,1]]
    df[1] = df[1].apply(lambda x: x[0] if isinstance(x, list) else None)
    df = df.dropna(axis=0)
    df[0] = pd.to_datetime(df[0], unit='ms').reset_index(drop=True)
    df.columns = ['time', 'netflow']
    return df

df_ETH_Netflow = crypto_netflow("ETH")
df_BTC_Netflow = crypto_netflow("BTC")

df_BTC_Netflow


,time,netflow
0,2011-07-11,1.378733e+01
1,2011-07-18,-1.160835e+01
2,2011-07-25,1.333309e+01
3,2011-08-01,-3.846900e+00
4,2011-08-08,-1.094747e+01
...,...,...
707,2025-01-27,6.795580e+07
708,2025-02-03,1.324378e+09
709,2025-02-10,2.170963e+08
710,2025-02-17,1.012559e+09


### 稳定币发行量和净流入流出量

In [ ]:
# USDC, DAI, FDUSD, TUSD
# https://intel.arkm.com/dashboards/view?dashboardID=3c55c261-2e87-43c3-9748-3fbc95603b97

with open('./unprocessedFile/stablecoin_exchange_flow.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def stablecoin_netflow(token_name:str):

    df = pd.DataFrame(data[token_name])
    df = df[(df['inUSD']>0) | (df['outUSD']>0)]
    df = df.sort_values('time', ascending=True).reset_index(drop=True)
    df['netUSD'] = df['inUSD'] - df['outUSD']
    df['netValue'] = df['inValue'] - df['outValue']
    df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)
    df = df[['time','inUSD','outUSD','netUSD','inValue','outValue','netValue']]
    df.set_index('time', inplace=True)
    return df

df_usdc = stablecoin_netflow('usdc')
df_dai = stablecoin_netflow('dai')
df_fdusd = stablecoin_netflow('fdusd')
df_tusd = stablecoin_netflow('tusd')
df_tusd

,inUSD,outUSD,netUSD,inValue,outValue,netValue
time,,,,,,
2019-01-04,2.617724e+06,2.641860e+06,-2.413579e+04,2.628145e+06,2.652377e+06,-2.423188e+04
2019-01-05,9.868962e+05,2.869411e+06,-1.882515e+06,9.900286e+05,2.878519e+06,-1.888490e+06
2019-01-06,1.989521e+06,2.253595e+06,-2.640741e+05,1.998030e+06,2.263234e+06,-2.652036e+05
2019-01-07,8.236969e+05,3.363097e+06,-2.539400e+06,8.285775e+05,3.383024e+06,-2.554446e+06
2019-01-08,3.549698e+06,4.587520e+06,-1.037822e+06,3.566126e+06,4.608752e+06,-1.042625e+06
...,...,...,...,...,...,...
2025-03-07,4.521310e+04,5.473149e+04,-9.518395e+03,4.532864e+04,5.487136e+04,-9.542719e+03
2025-03-08,2.337379e+05,1.991500e+04,2.138229e+05,2.343206e+05,1.996466e+04,2.143560e+05
2025-03-09,1.007923e+05,6.693894e+04,3.385332e+04,1.011126e+05,6.715167e+04,3.396091e+04


In [ ]:
# All stablecoin's netflow in crypto market
# IntoTheBlock

with open('./unprocessedFile/allStablecoin_exchange_flows.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df_all_stablecoin = pd.DataFrame(data['data']['rows'])
df_all_stablecoin[0] = pd.to_datetime(df_all_stablecoin[0]).dt.tz_localize(None)
df_all_stablecoin.columns = ['time', 'inflow', 'outflow', 'netflow']
df_all_stablecoin = df_all_stablecoin.sort_values('time', ascending=True).reset_index(drop=True)
df_all_stablecoin.set_index('time', inplace=True)
df_all_stablecoin

,inflow,outflow,netflow
time,,,
2019-09-21,9.285368e+07,9.678754e+07,-3.933857e+06
2019-09-22,1.935425e+08,1.880211e+08,5.521311e+06
2019-09-23,3.890572e+08,3.527504e+08,3.630681e+07
2019-09-24,3.497708e+08,3.120993e+08,3.767146e+07
2019-09-25,2.019344e+08,2.074047e+08,-5.470372e+06
...,...,...,...
2025-03-08,7.036590e+08,7.376716e+08,-3.401259e+07
2025-03-09,1.049148e+09,9.061097e+08,1.430383e+08
2025-03-10,4.769477e+09,4.949395e+09,-1.799181e+08


### SPY, VIX, 美元指数

In [ ]:
# https://finance.yahoo.com/quote/%5EGSPC/
# https://finance.yahoo.com/quote/DX-Y.NYB/
# https://finance.yahoo.com/quote/%5EVIX/

from __future__ import annotations

import pandas as pd
import yfinance as yf

ticker_list = [
    '^GSPC',
    'DX-Y.NYB',
    '^VIX'
]

START_DATE="2018-01-01"
END_DATE="2025-03-14"
TIME_INTERVAL= "1D"

def download_data(
    ticker_list: list[str],
    start_date: str,
    end_date: str,
    time_interval: str,
    proxy: str | dict = None,
) -> pd.DataFrame:

    # Download and save the data in a pandas DataFrame
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    data_df = pd.DataFrame()
    for tic in ticker_list:
        temp_df = yf.download(
            tic,
            start=start_date,
            end=end_date,
            interval=time_interval,
            proxy=proxy,
        )
        temp_df.columns = temp_df.columns.droplevel(1)
        temp_df["tic"] = tic
        data_df = pd.concat([data_df, temp_df])

    return data_df

us_stock_data=download_data(ticker_list=ticker_list,start_date=START_DATE,end_date=END_DATE,time_interval=TIME_INTERVAL)
print(us_stock_data)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price             Close         High          Low         Open      Volume  \
Date                                                                         
2018-01-02  2695.810059  2695.889893  2682.360107  2683.729980  3397430000   
2018-01-03  2713.060059  2714.370117  2697.770020  2697.850098  3544030000   
2018-01-04  2723.989990  2729.290039  2719.070068  2719.310059  3697340000   
2018-01-05  2743.149902  2743.449951  2727.919922  2731.330078  3239280000   
2018-01-08  2747.709961  2748.510010  2737.600098  2742.669922  3246160000   
...                 ...          ...          ...          ...         ...   
2025-03-07    23.370001    26.559999    23.090000    24.850000           0   
2025-03-10    27.860001    29.559999    24.680000    24.700001           0   
2025-03-11    26.920000    29.570000    26.180000    27.940001           0   
2025-03-12    24.230000    26.910000    23.889999    26.879999           0   
2025-03-13    24.540001    25.290001    23.459999    24.920000  

In [183]:
df = pd.DataFrame(us_stock_data)
df = df.drop(columns=['High','Low','Open']).reset_index(drop=True)
df_spy = df[df['tic']=='^GSPC']
df_vix = df[df['tic']=='^VIX']
df_UsIndex = df[df['tic']=='DX-Y.NYB']
df_UsIndex

Price,Close,Volume,tic
1809,91.849998,0,DX-Y.NYB
1810,92.160004,0,DX-Y.NYB
1811,91.849998,0,DX-Y.NYB
1812,91.949997,0,DX-Y.NYB
1813,92.330002,0,DX-Y.NYB
...,...,...,...
3614,103.839996,0,DX-Y.NYB
3615,103.910004,0,DX-Y.NYB
3616,103.440002,0,DX-Y.NYB
3617,103.610001,0,DX-Y.NYB
